<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_07/class_007_image_processing_smart_trash_bin_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 281MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 224
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    img = cv2.imread(name)
    resized = cv2.resize(img, (Img_Size,Img_Size))
    X.append(resized)
m = len(Y)
print(n,m)

2527 2527


In [9]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [10]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [11]:
df=pd.DataFrame(label,columns=['target'])

In [12]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [13]:
y = pd.get_dummies(df['target'])
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
#y

In [14]:
#label

In [15]:
n =len(img_name)
data = np.empty((n, 224, 224, 3))

In [16]:
# https://pythontutorials.eu/deep-learning/transfer-learning/

In [17]:
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize

for im in X:
    im = preprocess_input(im)
    im = resize(im, output_shape=(224, 224))
    data[i] = im

In [18]:
n

2527

In [19]:
y.values

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [20]:
n =len(img_name)
m = len(tipos)

'''
labels_tf = np.empty(n, dtype=int)
for idx, yi in enumerate(label):
  labels_tf[idx] = yi
'''
idx = 0
labels_tf = np.empty([n,m], dtype=int)
for yi in y.values:
  labels_tf[idx] = yi
  idx = idx + 1



In [21]:
# check https://keras.io/api/applications/#mobilenetv2

In [22]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

D = len(tipos)
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(D, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# train the model on the new data for a few epochs
model.fit(x=data, y=labels_tf, epochs=20, verbose=2)

Epoch 1/20
79/79 - 14s - loss: 1.7902 - accuracy: 0.1820 - 14s/epoch - 172ms/step
Epoch 2/20
79/79 - 5s - loss: 1.7461 - accuracy: 0.2078 - 5s/epoch - 69ms/step
Epoch 3/20
79/79 - 5s - loss: 1.7319 - accuracy: 0.2125 - 5s/epoch - 69ms/step
Epoch 4/20
79/79 - 5s - loss: 1.7301 - accuracy: 0.2236 - 5s/epoch - 69ms/step
Epoch 5/20
79/79 - 6s - loss: 1.7283 - accuracy: 0.2311 - 6s/epoch - 70ms/step
Epoch 6/20
79/79 - 6s - loss: 1.7284 - accuracy: 0.2252 - 6s/epoch - 70ms/step
Epoch 7/20
79/79 - 6s - loss: 1.7292 - accuracy: 0.2307 - 6s/epoch - 72ms/step
Epoch 8/20
79/79 - 6s - loss: 1.7265 - accuracy: 0.2355 - 6s/epoch - 70ms/step
Epoch 9/20
79/79 - 6s - loss: 1.7268 - accuracy: 0.2315 - 6s/epoch - 70ms/step
Epoch 10/20
79/79 - 6s - loss: 1.7280 - accuracy: 0.2355 - 6s/epoch - 71ms/step
Epoch 11/20
79/79 - 6s - loss: 1.7270 - accuracy: 0.2355 - 6s/epoch - 71ms/step
Epoch 12/20
79/79 - 6s - loss: 1.7270 - accuracy: 0.2355 - 6s/epoch - 70ms/step
Epoch 13/20
79/79 - 6s - loss: 1.7263 - accura